<h2> Model's Overview </h2>

In [ ]:
# List of models with their specific token limits
models = [
    ("dmis-lab/biobert-v1.1", 512),
    ("microsoft/biogpt", 1024),
    ("allenai/longformer-base-4096", 4096),
    ("allenai/scibert_scivocab_uncased", 512)
]

<h3>BERT: </h3>
Utilizes a bidirectional Transformer architecture, meaning it processes the input text in both directions simultaneously. This allows BERT to capture the context around each word, considering all the words in the sentence.

<h3>GPT: </h3>
Employs a unidirectional Transformer architecture, processing the text from left to right. This design enables GPT to predict the next word in a sequence but limits its understanding of the context to the left of a given word.

<h4>Attention Heads and Layers:</h4> 
Both models use multi-head attention, but they differ in the number of layers and heads. BERT has two versions with different configurations, while GPT-1 has a 12-level, 12-headed structure.

<h4>Output Layer:</h4> 
BERT is fine-tuned with task-specific layers, while GPT-1 uses a linear-softmax layer for word prediction.

<h4>Use Case:</h4>
-> BERT is very good at solving sentence and token-level classification tasks. Extensions of BERT (e.g., sBERT) can be used for semantic search, making BERT applicable to retrieval tasks as well. Finetuning BERT to solve classification tasks is oftentimes preferable to performing few-shot prompting via an LLM.

->Encoder-only models such as BERT cannot generate text. This is where we need decoder-only models such as GPT. They are suitable for tasks like text generation, translation, etc.


# Comparison of Biomedical and Scientific NLP Model Architectures

| Model | Token Limit | Multi-Head Attention | No. of Attention Heads | No. of Layers | Vector Dimensionality |
|-------|-------------|----------------------|------------------------|----------------|------------------------|
| dmis-lab/biobert-v1.1 | 512 | Yes | 12 | 12 | 768 |
| microsoft/biogpt | 1024 | Yes | 24 | 24 | 2048 |
| allenai/longformer-base-4096 | 4096 | Yes | 12 | 12 | 768 |
| allenai/scibert_scivocab_uncased | 512 | Yes | 12 | 12 | 768 |

## Additional Notes:

1. **dmis-lab/biobert-v1.1**:
   - Based on BERT architecture
   - Specialized for biomedical text
   - Uses WordPiece tokenization

2. **microsoft/biogpt**:
   - Based on GPT architecture
   - Larger model compared to others in the list
   - Uses byte-level BPE tokenization

3. **allenai/longformer-base-4096**:
   - Modified attention mechanism for long sequences
   - Uses a combination of sliding window and global attention
   - Based on RoBERTa architecture

4. **allenai/scibert_scivocab_uncased**:
   - Based on BERT architecture
   - Uses its own vocabulary (scivocab) tailored for scientific text
   - Trained on a large corpus of scientific publications

Key Observations:
- All models use multi-head attention, which is a fundamental component of transformer-based architectures.
- BioBERT and SciBERT have the smallest token limits (512), while Longformer and BigBird can handle much longer sequences (4096 tokens).
- BioGPT stands out with a larger number of attention heads, layers, and vector dimensionality, indicating a more complex model.
- Despite differences in specialization, Longformer, BigBird, BioBERT, and SciBERT share similar architectural properties (12 attention heads, 12 layers, 768-dimensional vectors).

<h3> WordPiece Tokenization </h3>

In [23]:
#Breaks words into smaller subwords.

#Whitespace Tokenization: The sentence is split by whitespace into individual tokens.

#Subword Splitting: Each token is checked against the vocabulary. If the token is not in the vocabulary, it is split further into smaller subwords.

from transformers import AutoTokenizer, BertTokenizerFast

tokenizer = AutoTokenizer.from_pretrained("dmis-lab/biobert-v1.1")

# Example sentence
text = "Gut microbiota influences enzyme metabolism."

# Tokenization
tokens = tokenizer.tokenize(text)
print("Tokens:", tokens)

# Convert tokens to IDs
token_ids = tokenizer.convert_tokens_to_ids(tokens)
print("Token IDs:", token_ids)

# Add special tokens (CLS and SEP)
inputs = tokenizer(text, return_tensors="pt")
print("Input IDs:", inputs["input_ids"])

Tokens: ['G', '##ut', 'micro', '##bio', '##ta', 'influences', 'enzyme', 'metabolism', '.']
Token IDs: [144, 3818, 17599, 25959, 1777, 7751, 8744, 20443, 119]
Input IDs: tensor([[  101,   144,  3818, 17599, 25959,  1777,  7751,  8744, 20443,   119,
           102]])


In [33]:
#WordPiece algorithm.
from transformers import BertTokenizerFast, AutoTokenizer

# Load SciBERT tokenizer
tokenizer = AutoTokenizer.from_pretrained("allenai/scibert_scivocab_uncased")

# Example input
text = "Gut microbiota influences enzyme metabolism."

# Tokenize the text
tokens = tokenizer.tokenize(text)
print("Tokens:", tokens)

# Convert tokens to IDs
token_ids = tokenizer.convert_tokens_to_ids(tokens)
print("Token IDs:", token_ids)

# Prepare input with padding and truncation
inputs = tokenizer(text, return_tensors="pt", padding=True, truncation=True, max_length=512)
print("Input IDs:", inputs["input_ids"])


Tokens: ['gut', 'microbiota', 'influences', 'enzyme', 'metabolism', '.']
Token IDs: [7881, 17697, 7794, 3823, 5009, 205]
Input IDs: tensor([[  102,  7881, 17697,  7794,  3823,  5009,   205,   103]])


<h3> Byte Pair Encoding Tokenisation</h3>

In [35]:
#BPE starts with individual characters and iteratively merges the most frequent pairs of tokens.

#The BPE algorithm used in this tokenizer splits rare or out-of-vocabulary words into subword units.
#If a word is present in the vocabulary as a whole, it will remain intact with the </w> suffix.
#Byte Pair Encoding

from transformers import BioGptTokenizer, AutoTokenizer

tokenizer = AutoTokenizer.from_pretrained("microsoft/biogpt")

# Example sentence
text = "Gut microbiota influences enzyme metabolism."

# Tokenize the text
tokens = tokenizer.tokenize(text)
print("Tokens:", tokens)

# Convert tokens to IDs
token_ids = tokenizer.convert_tokens_to_ids(tokens)
print("Token IDs:", token_ids)

# Add special tokens if needed (e.g., <|endoftext|>)
inputs = tokenizer(text, return_tensors="pt", padding=True, truncation=True)
print("Input IDs:", inputs["input_ids"])

Asking to truncate to max_length but no maximum length is provided and the model has no predefined maximum length. Default to no truncation.


Tokens: ['Gut</w>', 'microbiota</w>', 'influences</w>', 'enzyme</w>', 'metabolism</w>', '.</w>']
Token IDs: [22560, 4940, 2991, 439, 795, 4]
Input IDs: tensor([[    2, 22560,  4940,  2991,   439,   795,     4]])


In [25]:
#It uses BPE tokenization and handles both common words and subword splits effectively.
#The Ġ prefix indicates that a word follows a space in the original text. This helps the model maintain the context of word boundaries.
from transformers import LongformerTokenizerFast, AutoTokenizer

tokenizer = AutoTokenizer.from_pretrained("allenai/longformer-base-4096")

# Example input text
text = "Gut microbiota influences enzyme metabolism."

# Tokenize the input text
tokens = tokenizer.tokenize(text)
print("Tokens:", tokens)

# Convert tokens to IDs
token_ids = tokenizer.convert_tokens_to_ids(tokens)
print("Token IDs:", token_ids)

# Prepare inputs with padding and truncation
inputs = tokenizer(text, return_tensors="pt", padding=True, truncation=True)
print("Input IDs:", inputs["input_ids"])


Asking to truncate to max_length but no maximum length is provided and the model has no predefined maximum length. Default to no truncation.


Tokens: ['G', 'ut', 'Ġmicrobiota', 'Ġinfluences', 'Ġenzyme', 'Ġmetabolism', '.']
Token IDs: [534, 1182, 46009, 16882, 32834, 30147, 4]
Input IDs: tensor([[    0,   534,  1182, 46009, 16882, 32834, 30147,     4,     2]])


In [20]:
# from transformers import AutoTokenizer
# 
# # Load tokenizer
# tokenizer = AutoTokenizer.from_pretrained("allenai/scibert_scivocab_uncased")
# 
# # Print the tokenizer type
# print(type(tokenizer))


<class 'transformers.models.bert.tokenization_bert_fast.BertTokenizerFast'>


In [ ]:
#Model Info:
#token limit, parameters, multi headed attentio?, no. of attention head, no. of layers, vector dimensional space